In [47]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from scipy.optimize import minimize_scalar
import matplotlib.pyplot as plt
import seaborn as sns

In [48]:
df = pd.read_csv("../Credit Risk Analysis/Task 3 and 4_Loan_Data.csv")
df = df.drop(columns=["customer_id"])

In [49]:
n_buckets = 6

In [50]:
kmeans = KMeans(n_clusters=n_buckets, random_state=42)
df['fico_bucket'] = kmeans.fit_predict(df[['fico_score']])

c:\Users\josse\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [51]:
centroids = kmeans.cluster_centers_.flatten()
sorted_centroids = np.sort(centroids)
boundaries = [df['fico_score'].min()]
boundaries = [(sorted_centroids[i] + sorted_centroids[i + 1]) / 2 for i in range(len(sorted_centroids) - 1)]
boundaries.append(df['fico_score'].max())

In [52]:
n_buckets = len(boundaries) - 1
rating_map = {}

for i in range(n_buckets):
    rating = chr(ord('A') + i)  # A, B, C, etc.
    rating_map[rating] = {
        'min_score': boundaries[i],
        'max_score': boundaries[i + 1],
        'bucket_index': i
    }

In [53]:
scores = None

In [54]:
if scores is None:
    scores = df['fico_score'].tolist()

ratings = []
n_buckets = len(boundaries) - 1

for score in scores:
    rating_assigned = False
    for i in range(n_buckets):
        if i == n_buckets - 1:  # Last bucket includes upper bound
            if boundaries[i] <= score <= boundaries[i + 1]:
                ratings.append(chr(ord('A') + i))
                rating_assigned = True
                break
        else:
            if boundaries[i] <= score < boundaries[i + 1]:
                ratings.append(chr(ord('A') + i))
                rating_assigned = True
                break
    
    if not rating_assigned:
        # Handle edge cases
        if score < boundaries[0]:
            ratings.append('A')
        else:
            ratings.append(chr(ord('A') + n_buckets - 1))

In [55]:
n_buckets = len(boundaries) - 1

for i in range(n_buckets):
    mask = (df['fico_score'] >= boundaries[i]) & (df['fico_score'] < boundaries[i + 1])
    if i == n_buckets - 1:  # Last bucket includes upper bound
        mask = (df['fico_score'] >= boundaries[i]) & (df['fico_score'] <= boundaries[i + 1])

    bucket_scores = df['fico_score'][mask]
    rating = chr(ord('A') + i)
    
    print(f"\nBucket {rating} [{boundaries[i]:.1f}, {boundaries[i + 1]:.1f}]:")
    print(f"  Count: {len(bucket_scores)}")
    print(f"  Mean FICO: {bucket_scores.mean():.1f}")
    print(f"  Std FICO: {bucket_scores.std():.1f}")
    
    if df['default'] is not None:
        bucket_defaults = df['default'][mask]
        default_rate = bucket_defaults.mean()
        print(f"  Default rate: {default_rate:.3f}")



Bucket A [545.1, 595.3]:
  Count: 1761
  Mean FICO: 574.2
  Std FICO: 13.9
  Default rate: 0.315

Bucket B [595.3, 635.5]:
  Count: 2396
  Mean FICO: 616.5
  Std FICO: 11.4
  Default rate: 0.193

Bucket C [635.5, 675.0]:
  Count: 2480
  Mean FICO: 654.5
  Std FICO: 11.3
  Default rate: 0.124

Bucket D [675.0, 721.8]:
  Count: 1914
  Mean FICO: 695.4
  Std FICO: 13.2
  Default rate: 0.076

Bucket E [721.8, 850.0]:
  Count: 798
  Mean FICO: 748.2
  Std FICO: 23.5
  Default rate: 0.030
